<a href="https://colab.research.google.com/github/sean-halpin/ml_tweepy_proj/blob/main/twitter_sentiment_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [2]:
BEARER_TOKEN = ""

In [13]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [3]:
import tweepy
import pandas as pd

client = tweepy.Client(
    bearer_token=BT
)

In [53]:
search_result = client.search_recent_tweets(query="english premier league -is:retweet",max_results=100)

In [54]:
list_of_lists = []
i=0
for sr in search_result.data:
  i+=1
  text = sr.text
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]

  list_of_lists.append([sr.text, str(labels[ranking[0]])])

In [ ]:
cols = ["tweet", "sentiment"]
df = pd.DataFrame(list_of_lists, columns=cols)
df.head()

In [56]:
df['sentiment']. value_counts()

neutral     52
negative    28
positive    20
Name: sentiment, dtype: int64

In [43]:
df.to_csv("tweets_annotated.csv", header=cols, index=False)